# STEP1

1.   資料蒐集(爬蟲)
2.   所有1~3月本益比平均or當季本益比 的 describe()-刪除極端值








### 匯入套件

In [1]:
pip install jmespath

In [2]:
import requests
import json
import jmespath
import pandas as pd
pd.set_option('precision', 5) ### 小數點後第五位
pd.set_option('display.max_column', None) ### 顯示所有欄
pd.set_option('display.max_rows', None) ### 顯示所有列
import time
import re
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook() ### 直接顯示互動式圖表
import warnings
warnings.simplefilter(action='ignore') ### 忽略警告

### 目標資訊

In [3]:
# 目標股票代碼
stocklist=["1201","1203","1210","1213","1215","1216","1217","1218","1219","1220","1225","1227","1229","1231","1232","1233","1234","1235","1236","1256","1702","1737"]
# 目標時間
datelist=[]
for year in range(2019,2022):
  for month in range(1,13):
    date=str(year)+str("%02d" % month)+"01"
    if date == '20210401':
      break
    else:
      datelist.append(date)

### 爬蟲(本益比資訊)

In [4]:
def get_peratio(stock,date):
  # 創建存放資訊dict
  data={"STOCK":[],"年分":[],"本益比":[]} 

  for date in datelist:
    try:
      for stock in stocklist:
        # 替代成所需的股票代碼及時間，後續爬蟲可直接使用
        request_url = f'https://www.twse.com.tw/exchangeReport/BWIBBU?response=json&date={date}&stockNo={stock}' 
        res = requests.get(request_url)
        resinfo = json.loads(res.text)
        # 定位json中所需的資訊
        peratio = jmespath.search('data[:][3][]',resinfo) 
        for peratioidx in range(len(peratio)):
          # 若該欄位並無放置資訊或資訊非浮點數格式出錯，因此遇到錯誤則跳過，直接刪除該股票代碼
          try:
            # 將資訊填入dict相對應位置
            data['本益比'].append(float(peratio[peratioidx]))
            data['STOCK'].append(stock)
            data['年分'].append((re.findall(r'^2\w{3}',date)[0]))
          except TypeError:
            print(peratioidx)
            pass
        # 避免過快取得網頁資訊會被封鎖ip
        time.sleep(3)
    except ValueError:
      stocklist.remove(stock)
  return data

In [5]:
df = pd.DataFrame(get_peratio(stocklist,datelist))
# 將欄位整理成以股票及年為群組，分別使用max﹑min及mean的方式
df1 = pd.concat([df.groupby(['STOCK','年分']).mean(), df.groupby(['STOCK','年分']).min(), df.groupby(['STOCK','年分']).max()], axis=1)
df1.columns = ['平均本益比','最小本益比','最大本益比']

In [6]:
# 爬蟲取得的本益比資訊
df1.head(5)

平均本益比  最小本益比  最大本益比
STOCK 年分                          
1201  2019   8.28202   6.08  14.48
      2020  26.83167   6.37  45.59
      2021  27.75073  20.33  29.73
1203  2019  13.86603  12.96  14.84
      2020  16.17196  13.65  19.74

In [7]:
df1.to_csv('stockstep1.csv')
!gdown --id '1INiw55LxmkbWHSrq5hDPoPc-mAtQ4XEq'
df1 = pd.read_csv('stockstep1.csv', index_col=0)

Downloading...
From: https://drive.google.com/uc?id=1INiw55LxmkbWHSrq5hDPoPc-mAtQ4XEq
To: /content/stockstep1.csv
100% 2.38k/2.38k [00:00<00:00, 4.32MB/s]


# STEP 2

1.   刪除取一年中每月淨利為負的(若標準過高則再作修正)-刪除極端值

2.   算出合理區間
  *   Link：https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1201&SYEAR=2020&SSEASON=4&REPORT_ID=C





### 找出EPS

In [8]:
### 1~3->ABOUT EPS

# 下載所需csv
!gdown --id '1qyBFxEfEF9jDHrTvG4i5SlNWtdCr_soC'
!gdown --id '17iFuLmAL3jOocE0vMZKqMkERWQnTR85B'
!gdown --id '19bqdpigtGm1J2BsV3wJgoJP98ulWMzxv'

# 合併3個檔案
read = pd.read_csv('stock_1.csv')
read.to_csv('stockstep2_ori.csv')
for i in range(2, 4):
  read = pd.read_csv(f'stock_{i}.csv')
  read.to_csv('stockstep2_ori.csv', header=False, mode='a+') ### mode='a+'可讀可寫，最後方追加
df2 = pd.read_csv('stockstep2_ori.csv', index_col='Unnamed: 0')

Downloading...
From: https://drive.google.com/uc?id=1qyBFxEfEF9jDHrTvG4i5SlNWtdCr_soC
To: /content/stock_1.csv
100% 169k/169k [00:00<00:00, 1.48MB/s]
Downloading...
From: https://drive.google.com/uc?id=17iFuLmAL3jOocE0vMZKqMkERWQnTR85B
To: /content/stock_2.csv
100% 169k/169k [00:00<00:00, 1.49MB/s]
Downloading...
From: https://drive.google.com/uc?id=19bqdpigtGm1J2BsV3wJgoJP98ulWMzxv
To: /content/stock_3.csv
100% 153k/153k [00:00<00:00, 1.34MB/s]


In [9]:
# EPS未整理前資訊
df2.head(5)

,出表日期,年度,季別,公司代號,公司名稱,營業收入,營業成本,原始認列生物資產及農產品之利益（損失）,生物資產當期公允價值減出售成本之變動利益（損失）,營業毛利（毛損）,未實現銷貨（損）益,已實現銷貨（損）益,營業毛利（毛損）淨額,營業費用,其他收益及費損淨額,營業利益（損失）,營業外收入及支出,稅前淨利（淨損）,所得稅費用（利益）,繼續營業單位本期淨利（淨損）,停業單位損益,合併前非屬共同控制股權損益,本期淨利（淨損）,其他綜合損益（淨額）,合併前非屬共同控制股權綜合損益淨額,本期綜合損益總額,淨利（淨損）歸屬於母公司業主,淨利（淨損）歸屬於共同控制下前手權益,淨利（淨損）歸屬於非控制權益,綜合損益總額歸屬於母公司業主,綜合損益總額歸屬於共同控制下前手權益,綜合損益總額歸屬於非控制權益,基本每股盈餘（元）
0,110/05/15,108,4,1101,台泥,122783014,86872759,--,--,35910255,--,--,35910255,5753534,--,30156721,2476698,32633419,7178329,25455090,--,--,25455090,5024759,--,30479849,24211081,--,1244009,29178084,--,1301765,4.43
1,110/05/15,108,4,1102,亞泥,89347637,63746928,--,--,25600709,14392,0,25586317,3523141,--,22063176,6330006,28393182,6149229,22243953,--,--,22243953,-617304,--,21626649,17459673,--,4784280,17652536,--,3974113,5.56
2,110/05/15,108,4,1103,嘉泥,1884002,1685405,--,--,198597,--,--,198597,633383,--,-434786,1420557,985771,167261,818510,590161,--,1408671,3141141,--,4549812,1297473,--,111198,4326485,--,223327,2.02
3,110/05/15,108,4,1104,環泥,5005731,4418966,--,--,586765,--,--,586765,415870,--,170895,1027783,1198678,56996,1141682,--,--,1141682,-183256,--,958426,1135477,--,6205,952128,--,6298,1.74
4,110/05/15,108,4,1108,幸福,3731696,3432815,--,--,298881,--,--,298881,225119,4120,77882,-23968,53914,8010,45904,0,--,45904,48035,--,93939,45905,--,-1,93940,--,-1,0.11


In [10]:
# 利用上方剩下的股票代碼刪除新csv中不需要的股票代碼
stocklist = [str(i) for i in list(set(df1.index.get_level_values('STOCK')))] ### get_level_values()獲取(多層)索引內容
df2 = df2[df2['公司代號'].isin(stocklist)]

# 刪除取一年中每月淨利為負的
df2 = df2[[x not in list(set(df2[df2['基本每股盈餘（元）']<0]['公司代號'].tolist())) for x in df2['公司代號']]]

In [11]:
# 資料為兩年即一季的每股盈餘，因此加總/2.25
df2 = df2.groupby(['公司代號']).sum()/2.25
df2 = df2.drop(df2.columns[:-1], axis=1)
# 確保每張資料表欄位名稱相同
df2 = df2.rename(columns={'公司代號': 'STOCK','基本每股盈餘（元）': 'EPS'})
df2.index.names = ['STOCK']

In [12]:
# EPS整理後資訊
df2.head(5)

,EPS
STOCK,
1201,1.73333
1203,1.90222
1210,3.52444
1215,5.81778
1216,3.59556


### 算出合理區間

In [13]:
# 以STOCK欄位合併兩張資料表
df = pd.merge(df1,df2,on='STOCK') 
df = pd.concat([df.groupby(['STOCK']).min().filter(['STOCK','最小本益比'], axis=1), df.groupby(['STOCK']).max().filter(['STOCK','最大本益比'], axis=1), df.groupby(['STOCK']).mean().filter(['STOCK','平均本益比','EPS'], axis=1)], axis=1)

In [14]:
# 算出股價區間
df['便宜股價'] = df['最小本益比']*df['EPS']
df['安全股價'] = ((df['最小本益比']+df['平均本益比'])/2)*df['EPS']
df['合理股價'] = df['平均本益比']*df['EPS']
df['觀察股價'] = ((df['最大本益比']+df['平均本益比'])/2)*df['EPS']
df['昂貴股價'] = df['最大本益比']*df['EPS']

In [15]:
df.filter(df.columns.tolist()[4:9], axis=1).to_csv('stockstep2.csv', encoding='big5') ### filter從原有df中選取出特定欄位製成新df
!gdown --id '1sARbyWVvtU_oTujMYAB58B4_9XrdnFjM'
df2 = pd.read_csv('stockstep2.csv', index_col=0, encoding='big5')

Downloading...
From: https://drive.google.com/uc?id=1sARbyWVvtU_oTujMYAB58B4_9XrdnFjM
To: /content/stockstep2.csv
100% 1.03k/1.03k [00:00<00:00, 1.58MB/s]


In [16]:
# 股價區間
df2.head(5)

,便宜股價,安全股價,合理股價,觀察股價,昂貴股價
STOCK,,,,,
1201,10.53867,23.43017,36.32167,57.67217,79.02267
1203,24.65280,27.79590,30.93901,35.53795,40.13689
1210,38.55742,44.24217,49.92692,57.00066,64.07440
1215,59.22498,67.85241,76.47984,100.28652,124.09320
1216,36.81849,54.97704,73.13558,82.62686,92.11813


# STEP3


1.   各項指標評分


In [17]:
# 收盤價及合理股價差異(按照高低估的排序0~30分，差異越大或取分數越高)
!gdown --id '1vD3StawgkocwC8YSXT9CVmq7WDV0UixV'

usecol = ['證券代號','收盤價']

read = pd.read_csv('stock_12.csv', index_col=0, header=2, usecols=usecol) ### usecols只選取出所需欄位
stocklist = [str(i) for i in list(set(df2.index.get_level_values('STOCK')))]
df3_4 = read[read.index.isin(stocklist)]
df3_4.index.names = ['STOCK']
df3_4.index = df3_4.index.astype(int)
df3_4 = pd.concat([df2, df3_4], axis=1)

df3_4['diff']= df3_4['合理股價'] - df3_4['收盤價']
# 最高分38，調整成30
df3_4['SCORE_4']= (df3_4['合理股價'] - df3_4['收盤價']).rank()*2*(30/38)

Downloading...
From: https://drive.google.com/uc?id=1vD3StawgkocwC8YSXT9CVmq7WDV0UixV
To: /content/stock_12.csv
100% 2.85k/2.85k [00:00<00:00, 4.84MB/s]


In [18]:
# 收盤價及合理股價評分
df3_4.head(5)

,便宜股價,安全股價,合理股價,觀察股價,昂貴股價,收盤價,diff,SCORE_4
STOCK,,,,,,,,
1201,10.53867,23.43017,36.32167,57.67217,79.02267,21.55,14.77167,28.42105
1203,24.65280,27.79590,30.93901,35.53795,40.13689,38.20,-7.26099,4.73684
1210,38.55742,44.24217,49.92692,57.00066,64.07440,54.60,-4.67308,6.31579
1215,59.22498,67.85241,76.47984,100.28652,124.09320,76.60,-0.12016,14.21053
1216,36.81849,54.97704,73.13558,82.62686,92.11813,73.00,0.13558,15.78947


In [19]:
# 4~6->ABOUT GrossMargin & OperatingProfitMargin & NetProfitMargin

# 下載所需csv
!gdown --id '1-PIx_K_r8-akJYtVYnJ6fo8WQ_Qv1r-H'
!gdown --id '1bh4q9bEDFVFxEId50ovllExZmtl17EdW'
!gdown --id '1Bo4uoGEMM_MXKm87GPfhwA2bk4O5BjVy'

usecol = ['公司代號','毛利率(%)(營業毛利)/(營業收入)','營業利益率(%)(營業利益)/(營業收入)','稅後純益率(%)(稅後純益)/(營業收入)']

# 合併3個檔案
read = pd.read_csv('stock_4.csv', index_col=0, usecols=usecol)
for i in range(5,7):
  read1 = pd.read_csv(f'stock_{i}.csv', index_col=0, usecols=usecol)
  read = pd.merge(read,read1,on='公司代號')

df3_1 = read[read.index.isin(stocklist)]
df3_1.index.names = ['STOCK']
# 新增分數欄位
df3_1['SCORE_1']=0

Downloading...
From: https://drive.google.com/uc?id=1-PIx_K_r8-akJYtVYnJ6fo8WQ_Qv1r-H
To: /content/stock_4.csv
100% 40.5k/40.5k [00:00<00:00, 1.28MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bh4q9bEDFVFxEId50ovllExZmtl17EdW
To: /content/stock_5.csv
100% 41.1k/41.1k [00:00<00:00, 1.30MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Bo4uoGEMM_MXKm87GPfhwA2bk4O5BjVy
To: /content/stock_6.csv
100% 40.7k/40.7k [00:00<00:00, 1.28MB/s]


In [20]:
# 毛利率&營業利益率&稅後利益率
df3_1.head(5)

,毛利率(%)(營業毛利)/(營業收入)_x,營業利益率(%)(營業利益)/(營業收入)_x,稅後純益率(%)(稅後純益)/(營業收入)_x,毛利率(%)(營業毛利)/(營業收入)_y,營業利益率(%)(營業利益)/(營業收入)_y,稅後純益率(%)(稅後純益)/(營業收入)_y,毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入),SCORE_1
STOCK,,,,,,,,,,
1201,28.07,1.13,0.65,29.19,3.19,2.87,29.20,2.19,1.94,0
1203,34.39,17.74,13.44,33.82,16.13,11.48,34.48,17.42,14.49,0
1210,14.78,4.50,4.04,15.02,5.16,4.99,13.83,3.85,4.08,0
1215,11.79,4.26,4.37,15.88,8.56,7.39,15.82,8.19,6.62,0
1216,34.28,6.65,6.13,34.43,6.66,6.89,33.90,6.43,6.75,0


In [21]:
# 7~9->ABOUT Volume
!gdown --id '13krszbw-j3a3lEOsnrohd62ErYa1KOSn'
!gdown --id '1OoYey4KXJaGPnYsGyWAh8R0ME5YscZ25'
!gdown --id '1NDmk6RNxqK9F7ea9mD4hfYpEZwicTUB5'

usecol = ['證  券  名  稱','成  交  量']

# 合併3個檔案
read = pd.read_csv('stock_7.csv', header=1, index_col=0, usecols=usecol)
read = read.iloc[16:38]
for i in range(8,10):
  read1 = pd.read_csv(f'stock_{i}.csv', header=1, index_col=0, usecols=usecol)
  read1 = read1.iloc[16:38]
  read = pd.merge(read,read1,on='證  券  名  稱')

# index包含其他不需要的資訊，使用re去除
read.index = read.index.to_series().apply(lambda x:re.findall(r'^1\w{3}',x)[0])

df3_2 = read[read.index.isin(stocklist)]
df3_2.index.names = ['STOCK']
df3_2.index = df3_2.index.astype(int)
# 新增分數欄位
df3_2['SCORE_2'] = 0
for idx in df3_2.columns[:-1]:
  df3_2[idx] = df3_2[idx].apply(lambda x:x.replace(",", ""))

Downloading...
From: https://drive.google.com/uc?id=13krszbw-j3a3lEOsnrohd62ErYa1KOSn
To: /content/stock_7.csv
100% 1.88M/1.88M [00:00<00:00, 8.78MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OoYey4KXJaGPnYsGyWAh8R0ME5YscZ25
To: /content/stock_8.csv
100% 1.85M/1.85M [00:00<00:00, 8.62MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NDmk6RNxqK9F7ea9mD4hfYpEZwicTUB5
To: /content/stock_9.csv
100% 1.94M/1.94M [00:00<00:00, 9.11MB/s]


In [22]:
# 成交量
df3_2.head(5)

,成 交 量_x,成 交 量_y,成 交 量,SCORE_2
STOCK,,,,
1201,24014773,14261072,29416915,0
1203,1575571,786770,2503088,0
1210,70817221,32352365,73226832,0
1215,23056028,8024792,20559313,0
1216,246989051,148377767,230142259,0


In [23]:
# 10.11->ABOUT ​DebtsRatio & QuickRatio & ReceivablesTurnoverRatio & InventoryTurnover & TotalAssetsTurnoverRatio
!gdown --id '1cCu1PyZHAQ-U7aWBzHoMCX_czuvaigOo'
!gdown --id '1200qFzf6D2hf1fBCLhvsEhsDt86AALFY'

usecol = ['公司代號','財務結構-負債佔資產比率(%)','償債能力-速動比率(%)','經營能力-應收款項週轉率(次)','經營能力-存貨週轉率(次)','經營能力-總資產週轉率(次)']
 
# 合併2個檔案
read = pd.read_csv('stock_10.csv', index_col=0, usecols=usecol)
read1 = pd.read_csv('stock_11.csv', index_col=0, usecols=usecol)
read = pd.merge(read,read1,on='公司代號')
 
df3_3 = read[read.index.isin(stocklist)]
df3_3.index.names = ['STOCK']
# 新增分數欄位
df3_3['SCORE_3'] = 0

Downloading...
From: https://drive.google.com/uc?id=1cCu1PyZHAQ-U7aWBzHoMCX_czuvaigOo
To: /content/stock_10.csv
100% 118k/118k [00:00<00:00, 1.04MB/s]
Downloading...
From: https://drive.google.com/uc?id=1200qFzf6D2hf1fBCLhvsEhsDt86AALFY
To: /content/stock_11.csv
100% 118k/118k [00:00<00:00, 1.04MB/s]


In [24]:
# 負債佔資產比率&速動比率&應收款項周轉率&存貨周轉率&總資產周轉率
df3_3.head(5)

,財務結構-負債佔資產比率(%)_x,償債能力-速動比率(%)_x,經營能力-應收款項週轉率(次)_x,經營能力-存貨週轉率(次)_x,經營能力-總資產週轉率(次)_x,財務結構-負債佔資產比率(%)_y,償債能力-速動比率(%)_y,經營能力-應收款項週轉率(次)_y,經營能力-存貨週轉率(次)_y,經營能力-總資產週轉率(次)_y,SCORE_3
STOCK,,,,,,,,,,,
1201,63.64,68.62,7.14,10.64,1.00,61.98,73.90,7.05,10.29,1.00,0
1203,34.05,153.51,12.93,2.79,0.68,32.35,178.50,12.76,2.59,0.64,0
1210,48.57,55.16,12.81,7.09,1.73,46.05,60.20,13.23,7.17,1.65,0
1215,53.74,77.46,9.1,12.48,1.23,55.32,67.25,10.25,12.75,1.16,0
1216,63.56,79.56,24.46,7.86,1.01,63.54,77.08,24.58,7.5,0.92,0


In [25]:
# 確保每df每欄位資料型態皆相同
for  idx in df3_1.columns:
  df3_1[idx] = pd.to_numeric(df3_1[idx])
for  idx in df3_2.columns:
  df3_2[idx] = pd.to_numeric(df3_2[idx])
for  idx in df3_3.columns:
  df3_3[idx] = pd.to_numeric(df3_3[idx])

In [26]:
# 平均成交量>1000(10分)
# def col1(df):
for idx in range(len(df3_2)):
  if (df3_2['成  交  量_x'].iloc[idx]+df3_2['成  交  量_y'].iloc[idx]+df3_2['成  交  量'].iloc[idx])/3>1000000:
    df3_2['SCORE_2'].iloc[idx] = df3_2['SCORE_2'].iloc[idx]+10
    # return df
# 負債佔資產比率<50%(10分)
# def col2(df):
for idx in range(len(df3_3)):
  if df3_3['財務結構-負債佔資產比率(%)_y'].iloc[idx]<50:
    df3_3['SCORE_3'].iloc[idx]= df3_3['SCORE_3'].iloc[idx]+10
    # return df
# 速動比率>80%(10分)
# def col3(df):
for idx in range(len(df3_3)):
  if df3_3['償債能力-速動比率(%)_y'].iloc[idx]>80:
    df3_3['SCORE_3'].iloc[idx] = df3_3['SCORE_3'].iloc[idx]+10
    # return df
# 近兩年應收款項週轉率呈上升趨勢(5分)
# def col4(df):
for idx in range(len(df3_3)):
  if df3_3['經營能力-應收款項週轉率(次)_y'].iloc[idx]-df3_3['經營能力-應收款項週轉率(次)_x'].iloc[idx]>0:
    df3_3['SCORE_3'].iloc[idx] = df3_3['SCORE_3'].iloc[idx]+5
    # return df
# 近兩年存貨週轉率呈上升趨勢(5分)
# def col5(df):
for idx in range(len(df3_3)):
  if df3_3['經營能力-存貨週轉率(次)_y'].iloc[idx]-df3_3['經營能力-存貨週轉率(次)_x'].iloc[idx]>0:
    df3_3['SCORE_3'].iloc[idx] = df3_3['SCORE_3'].iloc[idx]+5
    # return df
# 近兩年總資產週轉率>0.9(10分)
# def col6(df):
for idx in range(len(df3_3)):
  if df3_3['經營能力-總資產週轉率(次)_y'].iloc[idx]>0.9:
    df3_3['SCORE_3'].iloc[idx] = df3_3['SCORE_3'].iloc[idx]+5
    # return df
# 110年第一季毛利率>109年第四季毛利率(10分)
# def col7(df):
for idx in range(len(df3_1)):
  if df3_1['毛利率(%)(營業毛利)/(營業收入)'].iloc[idx]-df3_1['毛利率(%)(營業毛利)/(營業收入)_y'].iloc[idx]>0:
    df3_1['SCORE_1'].iloc[idx] = df3_1['SCORE_1'].iloc[idx]+10
    # return df
# 1.110年第一季營益率和毛利率差距<20%
# 2.110年第一季營益率比前一季營益率高(10分)
# def col8(df):
for idx in range(len(df3_1)):
  if df3_1['營業利益率(%)(營業利益)/(營業收入)'].iloc[idx]-df3_1['毛利率(%)(營業毛利)/(營業收入)'].iloc[idx]<20:
    if df3_1['營業利益率(%)(營業利益)/(營業收入)'].iloc[idx]-df3_1['營業利益率(%)(營業利益)/(營業收入)_x'].iloc[idx]>0:
        df3_1['SCORE_1'].iloc[idx] = df3_1['SCORE_1'].iloc[idx]+10
    # return df
# def col9(df):
# 110年第一季稅後純益率比109年第一季稅後純益率高1%以上，且須為正數(10分)
for idx in range(len(df3_1)):
  if df3_1['稅後純益率(%)(稅後純益)/(營業收入)'].iloc[idx]>0:
    if df3_1['稅後純益率(%)(稅後純益)/(營業收入)_x'].iloc[idx]>0:
      if df3_1['稅後純益率(%)(稅後純益)/(營業收入)'].iloc[idx]-df3_1['稅後純益率(%)(稅後純益)/(營業收入)_x'].iloc[idx]>0.01:
          df3_1['SCORE_1'].iloc[idx] = df3_1['SCORE_1'].iloc[idx]+10
    # return df
# df3_1 = col9(col8(col7(df3_1)))
# df3_2 = col1(df3_2)
# df3_3 = col6(col5(col4(col3(col2(df3_3)))))

In [27]:
# 毛利率&營業利益率&稅後利益率評分
df3_1.head(5)

,毛利率(%)(營業毛利)/(營業收入)_x,營業利益率(%)(營業利益)/(營業收入)_x,稅後純益率(%)(稅後純益)/(營業收入)_x,毛利率(%)(營業毛利)/(營業收入)_y,營業利益率(%)(營業利益)/(營業收入)_y,稅後純益率(%)(稅後純益)/(營業收入)_y,毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入),SCORE_1
STOCK,,,,,,,,,,
1201,28.07,1.13,0.65,29.19,3.19,2.87,29.20,2.19,1.94,30
1203,34.39,17.74,13.44,33.82,16.13,11.48,34.48,17.42,14.49,20
1210,14.78,4.50,4.04,15.02,5.16,4.99,13.83,3.85,4.08,10
1215,11.79,4.26,4.37,15.88,8.56,7.39,15.82,8.19,6.62,20
1216,34.28,6.65,6.13,34.43,6.66,6.89,33.90,6.43,6.75,10


In [28]:
# 成交量評分
df3_2.head(5)

,成 交 量_x,成 交 量_y,成 交 量,SCORE_2
STOCK,,,,
1201,24014773,14261072,29416915,10
1203,1575571,786770,2503088,10
1210,70817221,32352365,73226832,10
1215,23056028,8024792,20559313,10
1216,246989051,148377767,230142259,10


In [29]:
# 負債佔資產比率&速動比率&應收款項周轉率&存貨周轉率&總資產周轉率評分
df3_3.head(5)

,財務結構-負債佔資產比率(%)_x,償債能力-速動比率(%)_x,經營能力-應收款項週轉率(次)_x,經營能力-存貨週轉率(次)_x,經營能力-總資產週轉率(次)_x,財務結構-負債佔資產比率(%)_y,償債能力-速動比率(%)_y,經營能力-應收款項週轉率(次)_y,經營能力-存貨週轉率(次)_y,經營能力-總資產週轉率(次)_y,SCORE_3
STOCK,,,,,,,,,,,
1201,63.64,68.62,7.14,10.64,1.00,61.98,73.90,7.05,10.29,1.00,5
1203,34.05,153.51,12.93,2.79,0.68,32.35,178.50,12.76,2.59,0.64,20
1210,48.57,55.16,12.81,7.09,1.73,46.05,60.20,13.23,7.17,1.65,25
1215,53.74,77.46,9.10,12.48,1.23,55.32,67.25,10.25,12.75,1.16,15
1216,63.56,79.56,24.46,7.86,1.01,63.54,77.08,24.58,7.50,0.92,10


In [30]:
df = pd.concat([df3_1.filter(['STOCK','SCORE_1'], axis=1), df3_2.filter(['SCORE_2'], axis=1), df3_3.filter(['SCORE_3'], axis=1), df3_4.filter(['SCORE_4'], axis=1)], axis=1).fillna(0)

In [31]:
df['FINAL_SCORE'] = df.sum(axis = 1)
df['FINAL_RANK'] = df['FINAL_SCORE'].rank(ascending=False) ### ascending分數越高排名越前面
df

,SCORE_1,SCORE_2,SCORE_3,SCORE_4,FINAL_SCORE,FINAL_RANK
STOCK,,,,,,
1201,30,10,5,28.42105,73.42105,5.0
1203,20,10,20,4.73684,54.73684,15.0
1210,10,10,25,6.31579,51.31579,17.0
1215,20,10,15,14.21053,59.21053,12.0
1216,10,10,10,15.78947,45.78947,19.0
1217,10,10,15,25.26316,60.26316,10.0
1218,10,10,25,7.89474,52.89474,16.0
1219,20,10,25,30.00000,85.00000,2.0
1220,30,10,20,20.52632,80.52632,4.0


In [36]:
df.to_csv('stockstep3.csv', encoding='big5')
result = pd.read_csv('stockstep3.csv', index_col=0, encoding='big5')
result.sort_values(by=['FINAL_RANK'])

,SCORE_1,SCORE_2,SCORE_3,SCORE_4,FINAL_SCORE,FINAL_RANK
STOCK,,,,,,
1232,30,10,35,23.68421,98.68421,1.0
1219,20,10,25,30.00000,85.00000,2.0
1225,30,10,15,26.84211,81.84211,3.0
1220,30,10,20,20.52632,80.52632,4.0
1201,30,10,5,28.42105,73.42105,5.0
1737,20,10,25,17.36842,72.36842,6.0
1233,30,0,20,22.10526,72.10526,7.0
1234,20,10,25,11.05263,66.05263,8.0
1231,30,10,20,3.15789,63.15789,9.0


In [35]:
df3_4.index = df3_4.index.map(str)
yinfo = df3_4['diff'].tolist()
xinfo = df3_4.index.get_level_values('STOCK').tolist()
# 輸出html檔
output_file("scatter.html")
tool = "hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select" ### 互動視覺圖功能
p = figure(x_range=xinfo, plot_width=750, plot_height=350, tools=tool, title="股價被高低估的程度", x_axis_label='股票代碼', y_axis_label='高低估程度')
p.circle(xinfo, yinfo, size=15, color="navy", alpha=0.8)
show(p)